Importing Spark Context and Spark Conf from pyspark,
Importing Spark Session from pyspark.sql,
Importing np from numpy, 
importing operator

In [1]:
from pyspark import SparkContext, SparkConf 
import numpy as np 
import operator

Creating a Spark Context with Application name and master core

In [2]:
conf=SparkConf().setAppName("Lab4-rdd").setMaster("local[*]")
sc = SparkContext(conf=conf)

Importing gzip, opening the cite75_99.txt gzip file and reading the gzip file and writing it to RDD.  

In [3]:
import gzip
with gzip.open('cite75_99.txt.gz', 'r') as f:
    rddCitations = sc.parallelize( f.readlines()[-200000:] )

Importing gzip, opening the patent gzip file and reading the gzip file and writing it to RDD.  

In [4]:
with gzip.open('apat63_99.txt.gz','r') as f:
    rddPatents = sc.parallelize( f.readlines()[-2000000:] ) 

Converting the rddCitation file in byte format to string format 

In [5]:
x = rddCitations.map(lambda x: x.decode('utf-8'))

Converting the rddPatents file in byte format to string format

In [6]:
y = rddPatents.map(lambda x: x.decode('utf-8'))

Splitting the string citation file by ',' and stripping of the leading and trailing white spaces

In [7]:
split_cit = x.map(lambda y: y.strip().split(','))

Splitting the string patent file by ',' and stripping of the leading and trailing white spaces, filtering out the null spaces present in the state column and considering only 1st and 5th column

In [8]:
split_pat = y.map(lambda x: x.strip().split(','))\
            .filter(lambda y: y[5] != '""')\
            .map(lambda y: (y[0], y[5]))


Splitting the string patent file by ',' and stripping of the leading and trailing white spaces to be taken into consideration at the last

In [9]:
split_pat1 = y.map(lambda x: x.strip().split(','))

Testing the display of the variable assigned above

In [10]:
split_pat1.take(4)

[['3997606',
  '1976',
  '6192',
  '1975',
  '"US"',
  '"IL"',
  '391745',
  '2',
  '2',
  '564',
  '1',
  '14',
  '10',
  '7',
  '1',
  '0.6122',
  '0.18',
  '4.1429',
  '3',
  '0.1111',
  '0.1',
  '0.1429',
  '0.1429'],
 ['3997607',
  '1976',
  '6192',
  '1975',
  '"NL"',
  '""',
  '540060',
  '3',
  '8',
  '564',
  '1',
  '14',
  '0',
  '0',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['3997608',
  '1976',
  '6192',
  '1975',
  '"US"',
  '"WI"',
  '475045',
  '2',
  '7',
  '564',
  '1',
  '14',
  '1',
  '6',
  '1',
  '0.7778',
  '0',
  '9',
  '1',
  '0',
  '0',
  '0',
  '0'],
 ['3997609',
  '1976',
  '6192',
  '1974',
  '"DE"',
  '""',
  '254145',
  '3',
  '7',
  '564',
  '1',
  '14',
  '3',
  '4',
  '0.6667',
  '0.625',
  '0',
  '11.75',
  '8.3333',
  '0.5',
  '0.3333',
  '0',
  '0']]

In [11]:
split_cit.take(1)

[['5991301', '5394398']]

Switching the citation and cited by using the lambda fucntion 

In [12]:
split_cited = split_cit.map(lambda y : (y[1],y[0]))

Joining split_cited database and split_pat database and storing it in variable cited_pat in RDD form.

In [13]:
cited_pat = split_cited.join(split_pat)

In [14]:
cited_pat.take (5)

[('5485455', ('5991301', '"NH"')),
 ('5485455', ('5996019', '"NH"')),
 ('5485455', ('5999541', '"NH"')),
 ('5485455', ('6002667', '"NH"')),
 ('5485455', ('6002689', '"NH"'))]

Sorting cited_pat by the second element and storing it in variable cited_pat_sort

In [15]:
cited_pat_sort = cited_pat.sortBy(lambda x : x[1])

Joining the cited_pat_sort with split_pat dataset as the data sets dervied above, sorting it by the Key and storing it in intermediate table

In [16]:
inter = cited_pat_sort.join(split_pat).sortBy(lambda x: x[0])

Taking the first values of intermediate table and displaying it

In [17]:
inter.take(5)

[('3997618', (('6008425', '"NJ"'), '"NJ"')),
 ('3997620', (('6005153', '"IL"'), '"IL"')),
 ('3997620', (('6008424', '"IL"'), '"IL"')),
 ('3997621', (('6002061', '"NJ"'), '"NJ"')),
 ('3997621', (('6004256', '"NJ"'), '"NJ"'))]

Filtering out the same state citation using the lambda function and keeping a count by the key.

In [18]:
final_count = inter.filter(lambda x : x[1][0][1] == x[1][1]).countByKey()

sorting the final by the count in descending order and storing it in variable count_final

In [28]:
count_final = sorted(final_count.items(), key=lambda x: x[1], reverse=True)

Converting the list to RDD datatset 

In [29]:
rddcount = sc.parallelize(count_final)

Displaying the top 10 counts in descending order for patents having same state citations

In [31]:
rddcount.take(10)

[('4683202', 16),
 ('4683195', 15),
 ('5572643', 14),
 ('4418068', 10),
 ('5333266', 10),
 ('5585479', 10),
 ('4133814', 9),
 ('4340563', 9),
 ('5313454', 9),
 ('5367328', 9)]

sorting the final by the count in ascending order and storing it in variable count_final1

In [32]:
count_final1 = sorted(final_count.items(), key=lambda x: x[1], reverse=False)

Converting the list to RDD datatset 

In [33]:
rddcount1 = sc.parallelize(count_final1)

Add the count to the patent database as mentioned in the sample output

In [35]:
split_pat1.join(rddcount1).collect()

[('3997936', ('1976', 1)),
 ('3998222', ('1976', 1)),
 ('3998233', ('1976', 1)),
 ('3998518', ('1976', 1)),
 ('3998768', ('1976', 1)),
 ('3999199', ('1976', 1)),
 ('3999410', ('1976', 3)),
 ('4000068', ('1976', 1)),
 ('4000953', ('1977', 1)),
 ('4000976', ('1977', 1)),
 ('4001559', ('1977', 1)),
 ('4001571', ('1977', 1)),
 ('4002289', ('1977', 1)),
 ('4002487', ('1977', 1)),
 ('4002607', ('1977', 1)),
 ('4003568', ('1977', 1)),
 ('4003660', ('1977', 1)),
 ('4003996', ('1977', 1)),
 ('4004035', ('1977', 1)),
 ('4004320', ('1977', 1)),
 ('4004371', ('1977', 1)),
 ('4004441', ('1977', 1)),
 ('4004681', ('1977', 1)),
 ('4005945', ('1977', 1)),
 ('4006271', ('1977', 1)),
 ('4006465', ('1977', 1)),
 ('4006686', ('1977', 1)),
 ('4006714', ('1977', 1)),
 ('4006736', ('1977', 3)),
 ('4006862', ('1977', 1)),
 ('4007293', ('1977', 1)),
 ('4007306', ('1977', 1)),
 ('4007613', ('1977', 3)),
 ('4007626', ('1977', 1)),
 ('4007866', ('1977', 1)),
 ('4008717', ('1977', 1)),
 ('4008783', ('1977', 1)),
 